In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate
import Utils
from Utils import Constants
import cv2
from facenet_pytorch import InceptionResnetV1
from Models import *
from DataLoaders import *

In [3]:
train_labels = pd.read_csv('train_data_clean.csv')
test_labels = pd.read_csv('test_data_clean.csv')
validation_labels = pd.read_csv('validation_data_clean.csv')
train_labels

,name,skin_tone,gender,age,is_face
0,TRAIN0001.png,0,0,1,False
1,TRAIN0002.png,5,1,0,True
2,TRAIN0005.png,1,1,0,False
3,TRAIN0007.png,1,0,1,True
4,TRAIN0009.png,7,0,1,False
...,...,...,...,...,...
6837,TRAIN9992.png,4,0,2,True
6838,TRAIN9993.png,1,1,1,True
6839,TRAIN9995.png,8,0,1,True
6840,TRAIN9998.png,4,1,1,False


In [4]:
def categorical_accuracy(ypred,y):
    #y is index, ypred i s one hot like in loss functions
    predicted = torch.argmax(ypred,1).long()
    correct = torch.mean((y.long() == predicted).float())
    return correct

def save_train_history(model,history,root=''):
    model_name = model.get_identifier()
    
    df = pd.DataFrame(history)
    df['model'] = model_name
    string = root + 'results/history_' + model_name + '.csv'
    df.to_csv(string,index=False)
    return df, string

def train_model(model,
                train_df,
                validation_df,
                root,
                epochs=300,
                lr=.001,
                batch_size=200,
                patience = 20,
                loss_weights = [2,1,.5],
                save_path=None,
                histogram =False,
                upsample=True,
                **kwargs,
               ):
    if save_path is None:
        save_path = root + 'models/'+ model.get_identifier()
        if upsample:
            save_path += '_balanced'
    if upsample:
        patience = int(patience/5) + 1
    train_loader = FaceGenerator(train_df,Constants.data_root,batch_size=batch_size,upsample=upsample,**kwargs)
    validation_loader = FaceGenerator(validation_df,Constants.data_root,validation=True,batch_size=batch_size,upsample=upsample,**kwargs)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train(True)
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    format_y = lambda y: y.long().to(device)
    
    def get_loss(m,xin,ytrue):
        outputs = m(xin)
        losses = [loss_fn(ypred.float(),format_y(y)) for y,ypred in zip(ytrue,outputs)]
        l1 = torch.mul(loss_weights[0],losses[0])
        l2 =  torch.mul(loss_weights[1],losses[1])
        l3 =  torch.mul(loss_weights[2],losses[2])
        total_losses = l1 + l2 + l3
        return outputs,total_losses
    
    def train_epoch():
        running_loss = 0
        running_accuracy = [0,0,0]
        curr_loss = 0
        count = 0
        for i, [x_batch, y_batch] in enumerate(train_loader):
            optimizer.zero_grad()
            if histogram:
                xh = torch_color_histogram(torch.clone(x_batch))
                xh = xh.to(device)
                xh.requires_grad_(True)
                xxb = x_batch.to(device)
                xxb.requires_grad_(True)
                xb = [xxb,xh]
            else:
                xb = x_batch.to(device)
                xb.requires_grad_(True)
            outputs,total_losses = get_loss(model, xb,y_batch)
            total_losses.backward()
            optimizer.step()
            running_loss += total_losses.item()
            print('curr loss',total_losses.item(), 'step',i,' | ',end='\r')
            count += 1
            with torch.no_grad():
                for i,(y,ypred) in enumerate(zip(y_batch,outputs)):
                    accuracy = categorical_accuracy(ypred.float(),format_y(y))
                    running_accuracy[i] += accuracy.item()
        return running_loss/count, [a/count for a in running_accuracy]
    
    def val_epoch():
        running_loss = 0
        running_accuracy = [0,0,0]
        count = 0
        for i, [x_batch, y_batch] in enumerate(validation_loader):
            if histogram:
                xh = torch_color_histogram(x_batch)
                xh = xh.to(device)
                xxb = x_batch.to(device)
                xb = [xxb,xh]
            else:
                xb = x_batch.to(device)
            outputs = model(xb)
            outputs,total_losses = get_loss(model,xb,y_batch)
            running_loss += total_losses.item()
            count += 1
            for i,(y,ypred) in enumerate(zip(y_batch, outputs)):
                accuracy = categorical_accuracy(ypred.float(),format_y(y))
                running_accuracy[i] += accuracy.item()
        return running_loss/count, [a/count for a in running_accuracy]
    
    
    best_val_loss = 100000
    steps_since_improvement = 0
    hist = []
    best_weights = model.state_dict()
    print('model being saved to',save_path)
    for epoch in range(epochs):
        print('epoch',epoch)
        model.train(True)
        avg_loss, avg_acc = train_epoch()
        print('train loss', avg_loss, 'train accuracy', avg_acc)
        model.train(False)
        val_loss, val_acc = val_epoch()
        print('val loss', val_loss, 'val accuracy', val_acc)
#         torch.save(model.state_dict(), save_path + '_epoch' + str(epoch))
        if best_val_loss > val_loss:
            torch.save(model,save_path)
            best_weights = model.state_dict()
            best_val_loss = val_loss
            steps_since_improvement = 0
        else:
            steps_since_improvement += 1
        
        hist_entry = {
            'epoch': epoch,
            'train_loss': avg_loss,
            'train_acc':avg_acc,
            'val_loss':val_loss,
            'val_acc': val_acc,
            'lr': lr,
            'loss_weights': '_'.join([str(l) for l in loss_weights])
        }
        hist.append(hist_entry)
        save_train_history(model,hist,root=root)
        if steps_since_improvement > patience:
            break
    return model,hist

m,h = train_model(
    DualHistogramModel(),
    train_labels,
    validation_labels,
    Constants.data_root,
    batch_size=100,
    histogram=True,
#     lr=10,
#     upsample=False,
)
del m
h

(35354, 5)
(7786, 5)
model being saved to ../../data/models/dual_dualfacenet_h400_st600_a400_g400_ed3_std2_ad2_gd2_hist10_histd_0.1_balanced
epoch 0


/opt/conda/envs/rapids/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train loss 6.1243552649762 train accuracy [0.18202447737593436, 0.4273697322202941, 0.5915892330435036]
val loss 5.934293661362085 val accuracy [0.2063357129884072, 0.5376267005235721, 0.7079576467856382]
epoch 1
train loss 5.976677040595793 train accuracy [0.2296066056452902, 0.47642392543076123, 0.6290834777604388]
val loss 5.774445081368471 val accuracy [0.2544782274426558, 0.5889266427510824, 0.7900059406573956]
epoch 2
train loss 5.934950789489315 train accuracy [0.2461278442356546, 0.48082338164081684, 0.6453243218741175]
val loss 5.761054662557749 val accuracy [0.2638759624499541, 0.615163973508737, 0.7640876372655233]
epoch 3
train loss 5.885675597325557 train accuracy [0.2653013108308706, 0.49117805812991944, 0.6567105935455042]
val loss 5.9273642637790775 val accuracy [0.2108884856869013, 0.5348509129805442, 0.7223792313001095]
epoch 4
train loss 5.878104886092709 train accuracy [0.27036931635127903, 0.49229963233241925, 0.6622661502011078]
val loss 5.756307626381899 val accu

[{'epoch': 0,
  'train_loss': 6.1243552649762,
  'train_acc': [0.18202447737593436, 0.4273697322202941, 0.5915892330435036],
  'val_loss': 5.934293661362085,
  'val_acc': [0.2063357129884072, 0.5376267005235721, 0.7079576467856382],
  'lr': 0.001,
  'loss_weights': '2_1_0.5'},
 {'epoch': 1,
  'train_loss': 5.976677040595793,
  'train_acc': [0.2296066056452902, 0.47642392543076123, 0.6290834777604388],
  'val_loss': 5.774445081368471,
  'val_acc': [0.2544782274426558, 0.5889266427510824, 0.7900059406573956],
  'lr': 0.001,
  'loss_weights': '2_1_0.5'},
 {'epoch': 2,
  'train_loss': 5.934950789489315,
  'train_acc': [0.2461278442356546, 0.48082338164081684, 0.6453243218741175],
  'val_loss': 5.761054662557749,
  'val_acc': [0.2638759624499541, 0.615163973508737, 0.7640876372655233],
  'lr': 0.001,
  'loss_weights': '2_1_0.5'},
 {'epoch': 3,
  'train_loss': 5.885675597325557,
  'train_acc': [0.2653013108308706, 0.49117805812991944, 0.6567105935455042],
  'val_loss': 5.9273642637790775,
  

In [5]:
DualHistogramModel()

DualHistogramModel(
  (base_model): InceptionResnetV1(
    (conv2d_1a): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2a): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2b): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_3b): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, aff

In [6]:
import torchvision